In [1]:
# Data processing
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

In [2]:
from sklearn.metrics import accuracy_score, confusion_matrix
def rates(true, pre, positives, negatives):
    tn, fp, fn, tp = confusion_matrix(true, pre).ravel()
    return accuracy_score(true, pre), tp / positives, tn / negatives

In [3]:

df = pd.read_csv('../../runs/3/home.csv')
df = df[:2100]
df = df[["RSSI"]]

window = 300
zeros = np.zeros(window)
ones = np.ones(window)
y = ones
l = len(ones) + window
f = True
while l <= 2100:
    if f:
        y = np.concatenate((y, zeros))
        f = False
    else:
        y = np.concatenate((y, ones))
        f = True
    l += window
num_steps = 500
df["y"] = y

In [4]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    X, y = list(), list()
    for i in range(x_data.shape[0]):
        end_ix = i + num_steps
        if end_ix >= x_data.shape[0]:
            break
        seq_X = x_data[i:end_ix]
        seq_y = y_data[end_ix]
        X.append(seq_X)
        y.append(seq_y)
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [5]:
X, y = lstm_data_transform(df["RSSI"].to_numpy(), y, num_steps=num_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [6]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

forest = LogisticRegression()
forest.fit(X_train, y_train)
predict = forest.predict(X_test)
print(rates(y_test, predict, sum(y_test), len(y_test) - sum(y_test)))

(0.99, 0.9934497816593887, 0.9853801169590644)
